<img src="https://openfisca.org/img/logo-openfisca.svg" width="198" align="right">
<h1 id="tocheading">Getting Started</h1>

This notebook illustrates the [key concepts](https://openfisca.org/doc/key-concepts/index.html) of OpenFisca that you will find in the documentation.

<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>


## Calculate a variable

Do some imports.

In [1]:
from pprint import pprint  # data pretty printer

from openfisca_core.simulation_builder import SimulationBuilder
from openfisca_france import FranceTaxBenefitSystem

Initialize the French [Tax and Benefit system](https://openfisca.org/doc/key-concepts/tax_and_benefit_system.html): it launches the entire legislation.

In [2]:
tax_benefit_system = FranceTaxBenefitSystem()

Initialize a test case: first step to give a persons situations for which you want to calculate taxes or benefits.  
Here we start with the persons: we have one parent and two children.

In [32]:
TEST_CASE = {
    'individus': {
        'parent1': {
            'age': {'2015-01': 30},
            'salaire_de_base': {'2024': 200000}
        },
        'enfant1': {
            'age': {'2015-01': 12}
        },
        'enfant2': {
            'age': {'2015-01': 18}
        }
    }
}

Define the role of those persons as members of `foyers_fiscaux` (tax households), `menages` (social households) and `familles` (families) groups.  
These groups come with the tax and benefit system.

In [47]:
TEST_CASE['foyers_fiscaux'] = {
    'foyer_fiscal1': {
        'declarants': ['parent1'],
        'personnes_a_charge': ['enfant1', 'enfant2'],
        'celibataire_ou_divorce': {'2024': True},
        'caseT': {'2024': True}
    }
}

TEST_CASE['menages'] = {
    'menage1': {
        'personne_de_reference': ['parent1'],
        'enfants': ['enfant1', 'enfant2']
    }
}

TEST_CASE['familles'] = {
    'famille1': {
        'parents': ['parent1'],
        'enfants': ['enfant1', 'enfant2']
    }
}

# display full test case
pprint(TEST_CASE)

{'familles': {'famille1': {'enfants': ['enfant1', 'enfant2'],
                           'parents': ['parent1']}},
 'foyers_fiscaux': {'foyer_fiscal1': {'caseT': {'2024': True},
                                      'celibataire_ou_divorce': {'2024': True},
                                      'declarants': ['parent1'],
                                      'personnes_a_charge': ['enfant1',
                                                             'enfant2']}},
 'individus': {'enfant1': {'age': {'2015-01': 12}},
               'enfant2': {'age': {'2015-01': 18}},
               'parent1': {'age': {'2015-01': 30},
                           'salaire_de_base': {'2024': 200000}}},
 'menages': {'menage1': {'enfants': ['enfant1', 'enfant2'],
                         'personne_de_reference': ['parent1']}}}


And use this test case to create a [simulation](https://openfisca.org/doc/key-concepts/simulation.html#simulation-the-framework-of-computation): the OpenFisca frame for calculating taxes or benefits.

In [48]:
simulation_builder = SimulationBuilder()
simulation = simulation_builder.build_from_entities(tax_benefit_system, TEST_CASE)

Calculate a variable: `af` (family allowance) for example.

In [49]:
print(simulation.calculate('ir_brut', '2024'))
print(simulation.calculate('ir_plaf_qf', '2024'))


[28629.2]
[40386.008]


In [21]:
pprint(tax_benefit_system.base_tax_benefit_system.variables)

{'aacc_defn': <4485676000_-4234538219475527648_non_salarie.aacc_defn object at 0x1284b3220>,
 'aacc_defs': <4485676000_-4234538219475527648_non_salarie.aacc_defs object at 0x1284b3b20>,
 'aacc_exon': <4485676000_-4234538219475527648_non_salarie.aacc_exon object at 0x12840a1c0>,
 'aacc_gits': <4485676000_-4234538219475527648_non_salarie.aacc_gits object at 0x1284b7ee0>,
 'aacc_impn': <4485676000_-4234538219475527648_non_salarie.aacc_impn object at 0x12843b760>,
 'aacc_imps': <4485676000_-4234538219475527648_non_salarie.aacc_imps object at 0x12843bd60>,
 'aacc_info': <4485676000_-4234538219475527648_non_salarie.aacc_info object at 0x128566160>,
 'aacc_pvce': <4485676000_-4234538219475527648_non_salarie.aacc_pvce object at 0x1285669a0>,
 'aacc_timp': <4485676000_6616628567343668264_ir.aacc_timp object at 0x129c38d00>,
 'aah': <4485676000_912101241184569639_aah.aah object at 0x12858adc0>,
 'aah_base': <4485676000_912101241184569639_aah.aah_base object at 0x128b42250>,
 'aah_base_non_cumula

In [24]:
tax_benefit_system.base_tax_benefit_system.variables['livret_epargne_populaire_taux']

<4485676000_-1612101762127943179_livret_epargne_populaire.livret_epargne_populaire_taux at 0x128c84f10>

## Test the impact of a reform

A reform represents a modified version of the real Tax and Benefit legislation* 

Reforms are gather in the module [`Reform`](https://github.com/openfisca/openfisca-france/tree/master/openfisca_france/reforms) of OpenFisca.

You have then to import the module extension corresponding to the reform of your concern.

Here we choose the fiscal reform of Landais, Piketty and Saez on the income tax ( http://www.revolution-fiscale.fr/la-reforme-proposee ). 

It is already implemented in OpenFisca under the name `landais_piketty_saez` .

In [17]:
from openfisca_france.reforms import landais_piketty_saez

ModuleNotFoundError: No module named 'openfisca_france.reforms'

Create a modified version of the tax and benefit system, affected by the changes introduced by the reform

In [8]:
reform = landais_piketty_saez.landais_piketty_saez(tax_benefit_system)

Create the test case you want to test.

In [9]:
REFORM_TEST_CASE = {
    'individus': {
        'parent1': {
            'age': {'2013-01': 40},
            'salaire_de_base': {'2013': 50000}
        }
    },
    'foyers_fiscaux': {
        'foyer_fiscal1': {
            'declarants': ['parent1']
        }
    },
    'menages': {    
        'menage1': {
            'personne_de_reference': ['parent1'],
        }
    },    
    'familles': {
        'famille1': {
            'parents': ['parent1'],
        }
    }
}

With the reform:

In [10]:
# Indicate that you want to perfom the reform on this test case
simulation_builder = SimulationBuilder()

# Simulate the reform
reform_simulation = simulation_builder.build_from_entities(reform, REFORM_TEST_CASE)

# Choose the variable you want to calcul : here the disposable income, "revenu_disponible"
reform_simulation.calculate('revenu_disponible', '2013')

array([14256.885], dtype=float32)

Without the reform: the baseline (counterfactual)

In [11]:
# Indicate that you want to perfom the standard system on the same test case
simulation_builder = SimulationBuilder()
baseline_simulation = simulation_builder.build_from_entities(tax_benefit_system, REFORM_TEST_CASE)

# Choose the variable you want to calcul
baseline_simulation.calculate('revenu_disponible', '2013')

array([34682.22], dtype=float32)

In [50]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

Parameters exploration and modification : https://openfisca.org/doc/coding-the-legislation/legislation_parameters.html
Entities : entities = [Individu, Famille, FoyerFiscal, Menage]
